# 天使人综合征与正常群体间的二分类模型

In [ ]:
import os
import numpy as np
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from tensorflow.keras.callbacks import Callback
from tensorflow import keras
from sklearn.metrics import f1_score, recall_score, precision_score

数据集路径

In [ ]:
TRAIN_SET_DIR = './dataset/train'
TEST_SET_DIR = './dataset/test'

构造generator，避免一次读取全部图片导致占满内存

In [ ]:
image_processor = ImageDataGenerator(validation_split=0.25)
train_generator = image_processor.flow_from_directory(
    directory=TRAIN_SET_DIR,
    target_size=(112, 96),
    batch_size=16,
    subset='training'
)
validation_generator = image_processor.flow_from_directory(
    directory=TRAIN_SET_DIR,
    target_size=(112, 96),
    batch_size=sum([
      len(os.listdir(os.path.join(TEST_SET_DIR, class_name)))
      for class_name in os.listdir(TEST_SET_DIR)
    ]),
    subset='validation'
)
test_generator = image_processor.flow_from_directory(
    directory=TEST_SET_DIR,
    target_size=(112, 96),
    batch_size=16
)

加载经过训练的预训练MobileFaceNet模型

In [ ]:
mobile_facenet = load_model('../../mobile_facenet.h5')

改造模型结构使其适应当前问题

In [ ]:
top_layer = Dense(
    2,
    kernel_initializer='he_normal',
    activation='softmax'
)(mobile_facenet.output)

In [ ]:
model = Model(inputs=mobile_facenet.input, outputs=top_layer)

增加checkpoint，保存中间结果

In [ ]:
model_checkpoint_callback = ModelCheckpoint(
        'saved_models/top_layer_trained_weights.{epoch:02d}-{val_accuracy:.2f}.h5',
        monitor='val_accuracy',
        mode='max',
        verbose=1,
        save_best_only=True
    )

In [ ]:
batches_per_epoch = train_generator.n // train_generator.batch_size

使用tensorboard

In [ ]:
tensorboard_callback = TensorBoard(
        log_dir='logs',
        batch_size=16
    )

监控准确率、查准率、查全率、auc

In [10]:
METRICS = [
  keras.metrics.TruePositives(name='tp'),
  keras.metrics.FalsePositives(name='fp'),
  keras.metrics.TrueNegatives(name='tn'),
  keras.metrics.FalseNegatives(name='fn'), 
  keras.metrics.BinaryAccuracy(name='accuracy'),
  keras.metrics.Precision(name='precision'),
  keras.metrics.Recall(name='recall'),
  keras.metrics.AUC(name='auc'),
]

编译模型

In [11]:
model.compile(
        loss='categorical_crossentropy',
        optimizer='adam',
        metrics = METRICS
    )

对模型进行训练

In [12]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=batches_per_epoch,
    epochs=30,
    callbacks=[
            # mae_callback,
            # early_stopping_callback,
            # metrics,
            model_checkpoint_callback,
            tensorboard_callback
        ],
    validation_data=validation_generator
)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/30
1/1 [==============================] - ETA: 0s - loss: 5.5175 - tp: 7.0000 - fp: 9.0000 - tn: 7.0000 - fn: 9.0000 - accuracy: 0.4375 - precision: 0.4375 - recall: 0.4375 - auc: 0.3867
Epoch 00001: val_accuracy improved from -inf to 0.50000, saving model to saved_models\top_layer_trained_weights.01-0.50.h5
1/1 [==============================] - 3s 3s/step - loss: 5.5175 - tp: 7.0000 - fp: 9.0000 - tn: 7.0000 - fn: 9.0000 - accuracy: 0.4375 - precision: 0.4375 - recall: 0.4375 - auc: 0.3867 - val_loss: 15.5724 - val_tp: 4.0000 - val_fp: 4.0000 - val_tn: 4.0000 - val_fn: 4.0000 - val_accuracy: 0.5000 - val_precision: 0.5000 - val_recall: 0.5000 - val_auc: 0.5000
Epoch 2/30
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
1/1 [==============================] - ETA: 0s - loss: 0.8970 - tp: 15.0000 - fp: 1.0000 - tn: 15.0000 - fn: 1.0000 - accuracy: 0.9375 - precision: 0.9375 - recal

打印模型结构

In [13]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 112, 96, 3)] 0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 114, 98, 3)   0           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 56, 48, 64)   1728        zero_padding2d[0][0]             
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 56, 48, 64)   256         conv2d[0][0]                     
_______________________________________________________________________________________

评估模型效果

In [14]:
model.evaluate_generator(test_generator, verbose=1)

Instructions for updating:
Please use Model.evaluate, which supports generators.
1/1 [==============================] - 0s 2ms/step - loss: 1.4462 - tp: 7.0000 - fp: 5.0000 - tn: 7.0000 - fn: 5.0000 - accuracy: 0.5833 - precision: 0.5833 - recall: 0.5833 - auc: 0.7292


[1.4462498426437378,
 7.0,
 5.0,
 7.0,
 5.0,
 0.5833333134651184,
 0.5833333134651184,
 0.5833333134651184,
 0.7291666865348816]

保存模型

In [15]:
model.save('./angelman-normal-cnn.h5')

In [18]:
best_model = load_model('./saved_models/top_layer_trained_weights.13-0.62.h5')

In [19]:
best_model.evaluate_generator(test_generator, verbose=1)

1/1 [==============================] - 0s 1ms/step - loss: 2.8321 - tp: 6.0000 - fp: 6.0000 - tn: 6.0000 - fn: 6.0000 - accuracy: 0.5000 - precision: 0.5000 - recall: 0.5000 - auc: 0.5000


[2.832077741622925, 6.0, 6.0, 6.0, 6.0, 0.5, 0.5, 0.5, 0.4999999701976776]